In [57]:
from time import sleep
from queue import Queue
from threading import Thread

import requests

import re
import string

import pandas as pd
import numpy as np

import transformers
import torch

from nltk import WordNetLemmatizer

import faiss

from tqdm.notebook import tqdm

from fake_useragent import UserAgent

# Загрузка и подготовка данных

In [3]:
data = pd.read_csv('../data/products.csv')
data = data.drop_duplicates()
tokenizer = transformers.BertTokenizer('../model/vocab.txt')
try:
	embedded_description = pd.read_csv('../data/embedded_description')
	embedded_product_composition = pd.read_csv('../data/embedded_product_composition')
	embedded_product_usage = pd.read_csv('../data/embedded_product_usage')
	embedded_3_in_1 = pd.read_csv('../data/embedded_3_in_1')
except:
	pass

# model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel,
													# transformers.DistilBertTokenizer,
													# 'distilbert-base-uncased')

data.info()

C:\Users\HardWorkingStation\AppData\Local\Temp\ipykernel_20168\1082297049.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/products.csv')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 40559 entries, 0 to 40579
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   40559 non-null  int64  
 1   sku                  40559 non-null  object 
 2   name                 40559 non-null  object 
 3   brand                40559 non-null  object 
 4   brand_type           40559 non-null  object 
 5   dimension17          37838 non-null  object 
 6   dimension18          39810 non-null  object 
 7   dimension19          9360 non-null   object 
 8   dimension20          9841 non-null   object 
 9   country              34488 non-null  object 
 10  price                40559 non-null  int64  
 11  currency             0 non-null      float64
 12  old_price            40559 non-null  int64  
 13  category_type        40555 non-null  object 
 14  url                  40559 non-null  object 
 15  images               40559 non-null 

Один товар может принадлежать нескольким категориям. Если бы у нас была информация о популярности товаров в той или иной категории, то можно было бы удалить товары из категорий, где они не пользуются спросом. Но т.к. такой информации у нас нет, просто удалим такие товары-дубликаты.

In [4]:
index_to_del = data[data.drop(['category', 'category_ru'], axis=1).duplicated()].index
display(f'Данных до удаления: {data.shape}')
data = data.drop(index_to_del)
display(f'Данных после удаления: {data.shape}')

'Данных до удаления: (40559, 28)'

'Данных после удаления: (34002, 28)'

In [5]:
def text_processing(text: str) -> str:
	# оставляем пропуски без изменений
	if text is np.nan:
		return np.nan
	# приводим текст к нижнему регистру
	text = text.lower()
	# заменяем символы и знаки пунктуации
	text = re.sub('\(.*?\)', '', text)
	trans_dict = str.maketrans('', '', string.punctuation)
	text = text.translate(trans_dict)
	# избавляемся от лишних пробелов
	text = ' '.join(text.split())

	return text

In [6]:
text_columns = ['description', 'product_usage', 'product_composition']

for column in text_columns:
	data[column] = data[column].apply(text_processing)

data.head()

,id,sku,name,brand,brand_type,dimension17,dimension18,dimension19,dimension20,country,...,main_product_sku,main_product_id,best_loyality_price,dimension29,dimension28,description,product_usage,product_composition,category,category_ru
0,203730,19000039636,03,Ecooking,standard,Жидкое мыло,Унисекс,NaN,NaN,Дания,...,19000039636,203730,NaN,False,False,нежное мыло,намочите руки нанесите на них мыло очистите ру...,aqua sodium laureth sulfate cocamidopropyl bet...,organika,органика
1,229474,19000031882,Anti-stress,Botavikos,standard,Сыворотки,Женский,Увлажнение и питание,Лицо,Россия,...,19000031882,229474,NaN,False,False,• пробуждает внутреннюю энергию клеток создава...,равномерно распределите на коже когда чувствуе...,aqua niacinamide glycerin gluconolactone xanth...,organika,органика
2,229480,19000031888,Dry oil,Botavikos,standard,Масло,Женский,NaN,Лицо,Россия,...,19000031888,229480,NaN,False,False,действие,встряхните перед использованием и распылите ма...,capryliccapric triglyceride olea europaea frui...,organika,органика
3,200485,19000046442,Catnip Chaser,Petstages,standard,игрушка для животных,NaN,NaN,NaN,США,...,19000046442,200485,NaN,False,False,игрушка трек с пластиковым мячиком тубом кошач...,подбирайте игрушки в соответствии с весом и дв...,пластик,tovary-dlja-zhivotnyh,товары для животных
4,202556,19000025382,SALT FACIAL SCRUB ORIGINAL,Kosette,standard,Скраб,Унисекс,Очищение,Лицо,NaN,...,19000025382,202556,NaN,False,True,нежный скраб,нанесите на чистую и влажную кожу затем аккура...,glycerin sea salt water silica cocoglucoside s...,azija,азия


In [7]:
def lower(text: str) -> str:
	if text is np.nan:
		return np.nan
	return text.lower()

In [8]:
columns_to_lower = ['name', 'brand', 'dimension17', 'dimension18', 'dimension19', 'dimension20', 'country', 'category_type']

for column in columns_to_lower:
	data[column] = data[column].apply(lower)

data.head()

,id,sku,name,brand,brand_type,dimension17,dimension18,dimension19,dimension20,country,...,main_product_sku,main_product_id,best_loyality_price,dimension29,dimension28,description,product_usage,product_composition,category,category_ru
0,203730,19000039636,03,ecooking,standard,жидкое мыло,унисекс,NaN,NaN,дания,...,19000039636,203730,NaN,False,False,нежное мыло,намочите руки нанесите на них мыло очистите ру...,aqua sodium laureth sulfate cocamidopropyl bet...,organika,органика
1,229474,19000031882,anti-stress,botavikos,standard,сыворотки,женский,увлажнение и питание,лицо,россия,...,19000031882,229474,NaN,False,False,• пробуждает внутреннюю энергию клеток создава...,равномерно распределите на коже когда чувствуе...,aqua niacinamide glycerin gluconolactone xanth...,organika,органика
2,229480,19000031888,dry oil,botavikos,standard,масло,женский,NaN,лицо,россия,...,19000031888,229480,NaN,False,False,действие,встряхните перед использованием и распылите ма...,capryliccapric triglyceride olea europaea frui...,organika,органика
3,200485,19000046442,catnip chaser,petstages,standard,игрушка для животных,NaN,NaN,NaN,сша,...,19000046442,200485,NaN,False,False,игрушка трек с пластиковым мячиком тубом кошач...,подбирайте игрушки в соответствии с весом и дв...,пластик,tovary-dlja-zhivotnyh,товары для животных
4,202556,19000025382,salt facial scrub original,kosette,standard,скраб,унисекс,очищение,лицо,NaN,...,19000025382,202556,NaN,False,True,нежный скраб,нанесите на чистую и влажную кожу затем аккура...,glycerin sea salt water silica cocoglucoside s...,azija,азия


Уберем лишние знаки из ссылок с изображениями

In [ ]:
def url_processing(text: str) -> str:
	return text.replace('"', '').replace('\'', '').replace('[', '').replace(']', '')

data['images'] = data['images'].apply(url_processing)

# Генерация эмбеддингов

In [ ]:
def lemmatization(data):
	if data is np.nan:
		return ''
	return ' '.join([WordNetLemmatizer().lemmatize(word) for word in data.split()])

Загрузим RuBERT для генерации эмбеддингов

In [9]:
config = transformers.BertConfig.from_json_file(
	'../model/bert_config.json')
model = transformers.BertModel.from_pretrained(
	'../model/pytorch_model.bin', config=config).to('cuda:0')

Some weights of the model checkpoint at ../model/pytorch_model.bin were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
batch_size = 1

for column in text_columns:
	text = data[column].fillna('')
	# lemmas = data[column].apply(lemmatization)
	vector = text.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512))
	# применим padding к векторам
	n = len(max(vector, key=len))
	# англ. вектор с отступами
	padded = np.array([i + [0]*(n - len(i)) for i in vector.values])

	# создадим маску для важных токенов
	attention_mask = np.where(padded != 0, 1, 0)

	embeddings = []
	for i in tqdm(range(padded.shape[0] // batch_size)):
		# преобразуем данные
		batch = torch.LongTensor(padded[batch_size*i : batch_size*(i+1)]).to('cuda:0')
		# преобразуем маску
		attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i : batch_size*(i+1)]).to('cuda:0')
		with torch.no_grad():
			batch_embeddings = model(batch, attention_mask=attention_mask_batch)

		# преобразуем элементы методом numpy() к типу numpy.array
		embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

	features = pd.DataFrame(np.concatenate(embeddings))
	features.to_csv(f'../data/embedded_{column}', index=False)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


  0%|          | 0/34002 [00:00<?, ?it/s]

  0%|          | 0/34002 [00:00<?, ?it/s]

  0%|          | 0/34002 [00:00<?, ?it/s]

In [11]:
batch_size = 1
text = []
# заменим пропуски в полях на пустую строку, для корректной генерации токенов
for d, pu, pc in data[['description', 'product_usage', 'product_composition']].values:
	if d is np.nan:
		d = ' '
	if pu is np.nan:
		pu = ' '
	if pc is np.nan:
		pc = ' '
	text.append(d + pu + pc)

data['3_in_1'] = text
text = data['3_in_1'].fillna('')
# lemmas = data['3_in_1'].apply(lemmatization)
vector = text.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512))
# применим padding к векторам
n = len(max(vector, key=len))
# англ. вектор с отступами
padded = np.array([i + [0]*(n - len(i)) for i in vector.values])

# создадим маску для важных токенов
attention_mask = np.where(padded != 0, 1, 0)

embeddings = []
for i in tqdm(range(padded.shape[0] // batch_size)):
	# преобразуем данные
	batch = torch.LongTensor(padded[batch_size*i : batch_size*(i+1)]).to('cuda:0')
	# преобразуем маску
	attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i : batch_size*(i+1)]).to('cuda:0')
	with torch.no_grad():
		batch_embeddings = model(batch, attention_mask=attention_mask_batch)

	# преобразуем элементы методом numpy() к типу numpy.array
	embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

features = pd.DataFrame(np.concatenate(embeddings))

features.to_csv(f'../data/embedded_3_in_1', index=False)

  0%|          | 0/34002 [00:00<?, ?it/s]

### Сформируем индексы faiss на основе эмбеддингов описания товаров

In [12]:
index = faiss.IndexFlatL2(embedded_description.shape[1])
print(index.ntotal)  # пока индекс пустой
index.add(np.ascontiguousarray(embedded_description.to_numpy().astype('float32')))
print(index.ntotal)  # теперь в нем n векторов

0
40559


Сохраним индексы в файл

In [13]:
faiss.write_index(index, 'faiss_description_index.index')

Получим предсказания по индексам

In [14]:
topn = 10
product_index_in_data = 6484
distances, same_embedding_indexes = index.search(np.ascontiguousarray(embedded_description.to_numpy().astype('float32')[product_index_in_data].reshape((1, -1))), 10)
print(same_embedding_indexes[0]) # индексы самых похожих векторов
print(distances) # расстояния, отсортированные по убыванию

[6484 6483 6486 6485 6482 6480 6479 6488 6478 6489]
[[ 0.       35.557343 47.900185 54.795876 56.23578  63.569435 66.96756
  77.014984 77.81093  80.93413 ]]


In [19]:
data.iloc[same_embedding_indexes[0]]

,id,sku,name,brand,brand_type,dimension17,dimension18,dimension19,dimension20,country,...,main_product_id,best_loyality_price,dimension29,dimension28,description,product_usage,product_composition,category,category_ru,3_in_1
6965,163525,11879-19000009028,cc water,erborian,standard,уходный праймер,унисекс,NaN,NaN,южная корея,...,163524,NaN,False,False,cc water крем для лица обогащенный экстрактом ...,наносите крем тонким слоем как любое средство ...,экстракт тигровой травы помогает успокоить кож...,ini-formaty,тревел-форматы,cc water крем для лица обогащенный экстрактом ...
6964,40849,17752,time plus longlasting make up,seventeen,standard,тональный крем,женский,NaN,NaN,греция,...,40852,NaN,False,False,тональный крем длительного действия,греческий бренд seventeen впервые появился на ...,NaN,makijazh,макияж,тональный крем длительного действиягреческий б...
6967,157834,19760343581,"драйв, бутылочный",бако текстиль,standard,косметички,женский,NaN,NaN,сша,...,157834,NaN,False,False,компактная косметичка выполнена из качественно...,чистка сухая и влажная стирка — только в исклю...,кожзам,odezhda-i-aksessuary,одежда и аксессуары,компактная косметичка выполнена из качественно...
6966,188602,19000033979,essential amino energy + electrolytes tangerin...,optimum nutrition,standard,NaN,унисекс,спортивное питание,NaN,NaN,...,188602,NaN,False,False,аминокислотный комплекс essential amino energy...,перорально,вес 285 г количество порций 30 в 1 порции 95 г...,lajfstajl,лайфстайл,аминокислотный комплекс essential amino energy...
6963,143062,10049-19760311514,color booster lip balm,artdeco,standard,бальзам,женский,увлажнение и питание,губы,германия,...,143059,NaN,False,False,питательный бальзам для губ color booster lip ...,нанесите бальзам на губы после нескольких прим...,ricinus communis seed oil diisostearyl malate ...,makijazh,макияж,питательный бальзам для губ color booster lip ...
6961,166256,19000008644,lunar,influence beauty,standard,пудровый хайлайтер,унисекс,NaN,NaN,россия,...,166256,NaN,False,False,мельчайшие сияющие частицы разных оттенков соз...,легко втушуйте хайлайтер пушистой кистью на вы...,mica hydrogenated polyisobutene talc silica tr...,makijazh,макияж,мельчайшие сияющие частицы разных оттенков соз...
6960,183522,19000028926,wrinkle reducer (red led) attachment,nuface,standard,массажер,унисекс,NaN,NaN,NaN,...,183522,NaN,False,False,wrinkle reducer – насадка к устройству nuface®...,применение возможно только с устройством nufac...,кэрол коул создатель nuface® начала свою карье...,tehnika,техника,wrinkle reducer – насадка к устройству nuface®...
6970,216994,101091-19000056470,neopro bubblegum leash,zee.dog,standard,поводок,NaN,NaN,NaN,сша,...,216993,NaN,False,False,коллекция neopro ™ — новейшая разработка zeedo...,поводок размера s предназначен для собак от 5 ...,прозрачный матовый полимер с цветной стропой в...,tovary-dlja-zhivotnyh,товары для животных,коллекция neopro ™ — новейшая разработка zeedo...
6959,216698,19000056250,neon coral keychain,zee.dog,standard,брелок для ключей,NaN,NaN,NaN,сша,...,216698,NaN,False,False,брелок для ключей собаковода это zeedog не бан...,универсальный брелок для любого ключа,полиэстер термопластичная резина закаленная сталь,tovary-dlja-zhivotnyh,товары для животных,брелок для ключей собаковода это zeedog не бан...
6971,205222,19000056712,issa 3 black,foreo,standard,зубная щетка,унисекс,очищение,для полости рта,швеция,...,205222,NaN,False,False,уход 4 в 1 для зубов десен языка и внутренней ...,как пользоваться,комплектация,tehnika,техника,уход 4 в 1 для зубов десен языка и внутренней ...


In [15]:
np.ascontiguousarray(embedded_description.to_numpy().astype('float32')[32487].reshape((1, -1)))

array([[ 3.24870000e+04,  1.61974296e-01,  1.69612780e-01,
        -2.48046070e-02, -1.85151160e-01,  6.80242181e-02,
        -3.98189947e-02,  3.38920020e-03,  9.63742137e-02,
        -1.03167415e-01,  2.92345345e-01, -2.02788800e-01,
         1.50223613e-01,  1.86988294e-01,  4.99615073e-02,
        -4.61208165e-01, -3.88336867e-01, -1.68155625e-01,
        -2.14272290e-01,  2.31765553e-01,  3.10934395e-01,
         2.36351117e-01,  7.13378191e-02,  3.44186664e-01,
         2.88002789e-02, -6.77370504e-02, -2.65579164e-01,
        -8.10857043e-02,  8.41881558e-02,  8.23946148e-02,
         2.33003125e-01,  2.32837319e-01,  3.47664565e-01,
        -2.17406166e-04,  2.55856842e-01,  2.38786325e-01,
         8.23035985e-02, -1.71225023e+00,  4.38627303e-01,
        -1.40812591e-01,  1.28130123e-01, -1.97826505e-01,
         3.89257133e-01, -1.10811241e-01,  6.96084723e-02,
         1.88111851e-03,  8.81056130e-01,  2.00314730e-01,
        -1.26266152e-01,  1.02331364e+00,  3.17612231e-0

In [58]:
def download_and_save_image(url: str):
	sleep(np.random.randint(3, 10))
	img_name = url.split('/')[-1]
	user_agent = UserAgent().random
	res = requests.get(url, timeout=3, headers={'User-Agent': user_agent})
	if res.status_code == 200:
		with open(f'../data/images/{img_name}', 'wb') as img:
			img.write(res.content)

In [34]:
urls = data.loc[800, 'images'].split(',')

In [38]:
urls[0].split('/')[-1]

'852578006836_1_sobzqsbm1f1nviuk.jpg'

In [59]:
download_and_save_image(urls[0])